# Load Libraries

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

## Load dataset

In [2]:
# Load the data

automobile_df = pd.read_csv("automobile.csv")

# no of rows and columns

automobile_df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [3]:
automobile_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')

In [4]:
# split x and y

x = automobile_df.iloc[0:,1:-1].values
y = automobile_df['mpg'].values

# expanding the dimension

y = np.expand_dims(y, -1)
x = np.expand_dims(y, -1)

In [5]:
# Build the model

model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
13/13 - 0s - loss: 246.9765
Epoch 2/100
13/13 - 0s - loss: 170.3806
Epoch 3/100
13/13 - 0s - loss: 109.8334
Epoch 4/100
13/13 - 0s - loss: 61.1251
Epoch 5/100
13/13 - 0s - loss: 28.8099
Epoch 6/100
13/13 - 0s - loss: 10.1176
Epoch 7/100
13/13 - 0s - loss: 2.3998
Epoch 8/100
13/13 - 0s - loss: 0.2656
Epoch 9/100
13/13 - 0s - loss: 0.0218
Epoch 10/100
13/13 - 0s - loss: 0.0330
Epoch 11/100
13/13 - 0s - loss: 0.0260
Epoch 12/100
13/13 - 0s - loss: 0.0173
Epoch 13/100
13/13 - 0s - loss: 0.0154
Epoch 14/100
13/13 - 0s - loss: 0.0154
Epoch 15/100
13/13 - 0s - loss: 0.0153
Epoch 16/100
13/13 - 0s - loss: 0.0152
Epoch 17/100
13/13 - 0s - loss: 0.0152
Epoch 18/100
13/13 - 0s - loss: 0.0151
Epoch 19/100
13/13 - 0s - loss: 0.0151
Epoch 20/100
13/13 - 0s - loss: 0.0150
Epoch 21/100
13/13 - 0s - loss: 0.0150
Epoch 22/100
13/13 - 0s - loss: 0.0149
Epoch 23/100
13/13 - 0s - loss: 0.0149
Epoch 24/100
13/13 - 0s - loss: 0.0148
Epoch 25/100
13/13 - 0s - loss: 0.0147
Epoch 26/100
13/13 - 0s -

In [6]:
# Make Predict
pred = model.predict(x)

# Calcaulte RMSE error.  RMSE is common for regression.
score = np.sqrt(mean_squared_error(pred,y))
print("Before saving model score (RMSE): {}".format(score))

Before saving model score (RMSE): 0.0904577119487219


## HDF5

In [7]:
# save entire network to HDF5 (save everything, suggested)
model.save("model_to_hdf5.h5")

# loading the model HDF5
from tensorflow.keras.models import load_model

model2 = load_model("model_to_hdf5.h5")
pred = model2.predict(x)

# Measure RMSE error.  RMSE is common for regression.

score = np.sqrt(mean_squared_error(pred,y))
print("After loading score (RMSE): {}".format(score))

After loading score (RMSE): 0.0904577119487219


## TensorFlow SavedModel

In [8]:
# save the model using TensorFlow SavedModel format
from tensorflow.keras.models import load_model

model.save('my_model')

# load the pb model
model_tf = load_model('my_model')

# predict
pred_tf = model_tf.predict(x)

score_tf = np.sqrt(mean_squared_error(pred_tf,y))
print("After loading score (RMSE): {}".format(score_tf))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model/assets
After loading score (RMSE): 0.0904577119487219


## JSON

In [9]:
# save model  structure to JSON (no weights)
model_json = model.to_json()
with open("model_to_json.json", "w") as json_file:
    json_file.write(model_json)

# saving the model weight seperatly
model.save_weights("j_model.h5")

In [10]:
# loading the model
from tensorflow.keras.models import model_from_json

with open('model_to_json.json', 'r') as json_file:
    loaded_model_json = json_file.read()

    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights("j_model.h5")

    print("Loaded model from disk")
    
    # predict
    pred_json = loaded_model.predict(x)

    score_json = np.sqrt(mean_squared_error(pred_json,y))
    print("After loading score (RMSE): {}".format(score_json))

Loaded model from disk
After loading score (RMSE): 0.0904577119487219


## YAML

In [11]:
# save model  structure to YAML (no weights)
model_yaml = model.to_yaml()
with open("model_to_yaml.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

# save the model weights separatly
model.save_weights('y_model.h5')

In [12]:
# loading the model
from tensorflow.keras.models import model_from_yaml

with open('model_to_yaml.yaml', 'r') as yaml_file:
    loaded_model_yaml = yaml_file.read()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    
    # load weights into new model
    loaded_model.load_weights("y_model.h5")

    print("Loaded model from disk")
    
    # predict
    pred_yaml = loaded_model.predict(x)

    pred_yaml = np.sqrt(mean_squared_error(pred_yaml,y))
    print("After loading score (RMSE): {}".format(pred_yaml))

Loaded model from disk
After loading score (RMSE): 0.0904577119487219


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/saving/model_config.py:88: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)
